In [1]:
import pandas as pd
import numpy as np
from scipy import io
import os, errno

In [2]:
# consolidate the true trials tDetect and exploration time across ALL the data.

direc1 = './dataFolders/Output/Step3_TrialLabelling/Labelled_Data_SVM/'
files = [file for file in os.listdir(direc1) if file.endswith('.csv')]


# load all the files with IRdtect data
direc2 = "./dataFolders/Output/Step4_IRDetect/"
IR_data_files = [file for file in os.listdir(direc2) if file.endswith('.npy')]

In [3]:
new_path = './dataFolders/Output/Step5_FilesWith_TrueTrialAnd_ProboscisDetect\\'

try:
    os.makedirs(new_path)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

for fname in files:
    df = pd.read_csv(direc1 + '\\' + fname)
    # pull out all the trueTrials and reset the index to default to get true trial #
    trueTrials = df[df.label == 1].loc[:, ['MothIN', 'MothOut']].reset_index(drop = True)
    
    # remove "cropped" from filename to match IR file
    if fname.find('_cropped') :
        new_fname = fname.replace('_cropped','')
    else: 
        new_fname = fname
    
    # load IR files and get indices of all the frames where IRDetect == 1
    IR_detect_file = [file for file in IR_data_files if new_fname[:-22] in file]
    
    loadedIRdetect = np.load(direc2 + '\\' + IR_detect_file[0])
    indx = np.where(loadedIRdetect)
    indx = indx[0].reshape(-1)
    
    # for each true trial, find the first frame where IRDetect == True
    proboscisDetect = []
    discoveryTime = []
    for framein, frameout in zip(trueTrials.MothIN, trueTrials.MothOut):
        allframes = [frame for frame in indx if framein < frame < frameout]
        if allframes == []:
            detect = np.nan
        else:
            detect = min(allframes)
        Time  = detect - framein
        proboscisDetect.append(detect)
        discoveryTime.append(Time)

    df1 = pd.DataFrame({'ProboscisDetect': proboscisDetect})
    df2 = pd.DataFrame({'DiscoveryTime' : discoveryTime})

    new_data = pd.concat([trueTrials, df1, df2], axis = 1)
    new_data.to_csv(new_path + fname[:-22] + '_RawDataForExplorationTime.csv')

IndexError: list index out of range

In [4]:
fname

'c-1_m10_MothInOut_labeled.csv'